In [64]:
from binance.cm_futures import CMFutures
import plotly.graph_objects as go
import requests
import json
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import datetime
api_url = "https://api.binance.com/api/v3/exchangeInfo"
response = requests.get(api_url)
cm_futures_client = CMFutures()
cm_futures_client = CMFutures("iPQMe46exUy10KiBaBahQT7ow1uzb9jaxlKj19Bg5BI8JEwJL5bw9LCvJtfKuVbP","J4FGJQdGPpNGm8jpnj3IuFuDJuAe7FIa3cM51L4z662UYux3qD0ByLJ0bfIltZvK")

# base endpoint https://fapi.binance.com
Most of the endpoints can be used in the testnet platform.<br>
The REST baseurl for testnet is "https://testnet.binancefuture.com"<br>
The Websocket baseurl for testnet is "wss://fstream.binancefuture.com"<br>



In [65]:
#TODO: Mümkün olan bütün trading pairlerin isimlerini al
#GET https://api.binance.com/api/v3/exchangeInfo


if response.status_code == 200:
    exchange_info = response.json()
    f = open("usdt_pairs.txt","a")
    for symbol in exchange_info['symbols']:
        if 'USDT' in symbol['symbol']:
            f.write(symbol['symbol'])
            f.write("\n")
    f.close()
else:
    print(f"Error: {response.status_code}")

In [66]:
def convert_to_df(data):
    data_df=pd.DataFrame(np.asarray(data,dtype=np.float64),columns=["Open Time","Open","High","Low","Close","Volume","Close Time","Quote Asset Volume","Number of Trades","Taker Buy Volume","Taker Buy Quote Asset Volume","Ignore"])
    data_df['Open Time'] = pd.to_datetime(data_df['Open Time'], unit='ms')
    data_df['Close Time'] = pd.to_datetime(data_df["Close Time"],unit='ms').dt.ceil("min")
    return data_df

In [67]:
data=convert_to_df(cm_futures_client.continuous_klines("BTCUSD",contractType="PERPETUAL", interval="15m"))

In [68]:
class Chart:
    def __init__(self,data,timeframe):
        self.data = data
        self.timeframe = timeframe
        self.candlesticks = go.Candlestick(x=self.data['Open Time'],
                open=self.data['Open'],
                high=self.data['High'],
                low=self.data['Low'],  
                close=self.data['Close'],name="Candlesticks")
        self.figure = go.Figure(self.candlesticks)
        
    def plot(self):
        self.figure.show()

    

    def add_line(self,line_name,*args):
        line = go.Scatter(x=[args[0], args[1]], y=[args[2], args[3]], mode='lines', line=dict(color='green'), name=line_name)
        self.figure.add_trace(line)

    def delete_line(self,line_name):
        self.figure.data = [trace for trace in self.figure.data if trace.name != line_name]
    
    #TODO: Gradient Descent fonksiyonu
    def gradient_descent(self):
        #0 slope 1 intercept
        linear_regression = LinearRegression()
        linear_regression.fit(self.data["Low"].index.values.reshape(-1,1),self.data["Low"])
        return [linear_regression.coef_[0], linear_regression.intercept_]
        
        

    #TODO: Swing High ve Swing Low bulma
    #Swingleri etraftaki kaç muma göre arayacağıma karar vermem lazım
    
        
    #TODO: Support ve Resistance Çizme
    

    #TODO: Trend Çizme
    



In [69]:
btc = Chart(data,"15m")

In [70]:
btc.gradient_descent()